In [4]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from PIL import Image
import os
from matplotlib import pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn import preprocessing 
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.neural_network import MLPRegressor
from sklearn.datasets import make_regression
from sklearn.datasets import make_moons
import cv2 as cv
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.optimizers import Adam


In [3]:
data_path = 'seg_train\seg_train'  
image_dimensions = (64, 64)  

def load_and_process_images(path):
    images = []  
    labels = []  
    for folder in os.listdir(path):
        folder_path = os.path.join(path, folder)  
        if os.path.isdir(folder_path):
            for file in os.listdir(folder_path):
                image_path = os.path.join(folder_path, file)  
                try:
                    image = Image.open(image_path)
                    image = image.resize(image_dimensions)
                    image = image.convert('L') 
                    images.append(np.array(image))
                    labels.append(folder)
                except Exception as e:
                    print("خطأ في قراءة الصورة:", image_path)
                    print(e)



    images = np.array(images)
    labels = np.array(labels)
    return images, labels

images, labels = load_and_process_images(data_path)


images = images.astype('float32') / 255.0

label_to_number = {label: number for number, label in enumerate(np.unique(labels))}
labels = np.array([label_to_number[label] for label in labels])
labels = to_categorical(labels)


In [5]:
number_of_labels = len(np.unique(labels))  # يجب أن يكون عدد الفئات المستهدفة

# إنشاء النموذج
model = Sequential()

# طبقة الإدخال والطبقة المخفية الأولى
model.add(Flatten(input_shape=(64, 64)))  # تحويل الصورة إلى مصفوفة أحادية الأبعاد
model.add(Dense(512, activation='relu'))  # طبقة مخفية بـ 512 عقدة

# إضافة طبقات مخفية إضافية (اختياري)
model.add(Dense(256, activation='relu'))  # طبقة مخفية إضافية بـ 256 عقدة

# طبقة الإخراج
model.add(Dense(number_of_labels, activation='softmax'))  # طبقة الإخراج بعدد عقد يساوي عدد الفئات

# تجميع النموذج
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# عرض ملخص النموذج
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 4096)              0         
                                                                 
 dense (Dense)               (None, 512)               2097664   
                                                                 
 dense_1 (Dense)             (None, 256)               131328    
                                                                 
 dense_2 (Dense)             (None, 2)                 514       
                                                                 
Total params: 2229506 (8.50 MB)
Trainable params: 2229506 (8.50 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [6]:
# # تدريب النموذج
# history = model.fit(images, labels, epochs=10, batch_size=32, validation_split=0.2)

# # طباعة نتائج التدريب
# print(history.history)

Epoch 1/10


ValueError: in user code:

    File "c:\Users\ASUS\anaconda3\Lib\site-packages\keras\src\engine\training.py", line 1377, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\ASUS\anaconda3\Lib\site-packages\keras\src\engine\training.py", line 1360, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\ASUS\anaconda3\Lib\site-packages\keras\src\engine\training.py", line 1349, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\ASUS\anaconda3\Lib\site-packages\keras\src\engine\training.py", line 1127, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "c:\Users\ASUS\anaconda3\Lib\site-packages\keras\src\engine\training.py", line 1185, in compute_loss
        return self.compiled_loss(
    File "c:\Users\ASUS\anaconda3\Lib\site-packages\keras\src\engine\compile_utils.py", line 277, in __call__
        loss_value = loss_obj(y_t, y_p, sample_wei

In [ ]:
clf = MLPClassifier(hidden_layer_sizes=(3, ), activation='tanh', solver='adam', 
                    alpha=0.0000001, batch_size=4, learning_rate='constant', learning_rate_init=0.005, 
                    power_t=0.5, max_iter=500, shuffle=True, random_state=11, tol=0.00001, 
                    verbose=True, warm_start=False, momentum=0.9, nesterovs_momentum=True, 
                    early_stopping=False, validation_fraction=0.2, 
                    beta_1=0.9, beta_2=0.999, epsilon=1e-08)